In [1]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
!pip install datasets transformers evaluate accelerate pandas numpy torch -q

In [3]:
from datasets import load_dataset
import os
import shutil

# Из предыдущей части кода
dataset = load_dataset("mteb/tweet_sentiment_extraction")

train_data = dataset["train"].shuffle(seed=42).select(range(500))
test_data = dataset["test"].shuffle(seed=42).select(range(200))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# --- 3. Data Preprocessing: Tokenization ---
print("\n--- Предобработка данных: Токенизация ---")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Определяем функцию токенизации
def tokenize_batch(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

# Применяем токенизацию ко всем наборам данных
train_tokenized_data = train_data.map(tokenize_batch, batched=True)
test_tokenized_data = test_data.map(tokenize_batch, batched=True)

# Переименовываем колонку 'label' в 'labels', так как Trainer ожидает 'labels'
train_tokenized_data = train_tokenized_data.rename_column("label", "labels")
test_tokenized_data = test_tokenized_data.rename_column("label", "labels")

# Удаляем оригинальные колонки, которые больше не нужны после токенизации
# ИЗМЕНЕНО ЗДЕСЬ: удален "selected_text"
train_tokenized_data = train_tokenized_data.remove_columns(["text", "label_text"])
test_tokenized_data = test_tokenized_data.remove_columns(["text", "label_text"])

# Преобразуем метки тональности из строк в числовые ID
# Как мы выяснили ранее, колонка 'label' (теперь 'labels') уже должна быть числовой (0, 1, 2).
# Поэтому следующий блок кода для map_labels_to_int скорее всего не нужен.
# Если у вас есть сомнения, выведите train_tokenized_data[0] после переименования,
# чтобы проверить тип данных в колонке 'labels'.
# Если 'labels' уже int, удалите или закомментируйте эти строки:
# label_map = {"positive": 0, "neutral": 1, "negative": 2}
# def map_labels_to_int(examples):
#     if isinstance(examples["labels"], str): # Проверяем, чтобы не было ошибки, если уже int
#         examples["labels"] = label_map[examples["labels"]]
#     return examples
# train_tokenized_data = train_tokenized_data.map(map_labels_to_int)
# test_tokenized_data = test_tokenized_data.map(map_labels_to_int)


print("\nПример токенизированных данных (первая запись, после преобразования):")
print(train_tokenized_data[0])
print(f"\nСтруктура токенизированного обучающего набора: {train_tokenized_data}")
print(f"Структура токенизированного тестового набора: {test_tokenized_data}")



--- Предобработка данных: Токенизация ---


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]


Пример токенизированных данных (первая запись, после преобразования):
{'id': 'e520773f6c', 'labels': 1, 'input_ids': [101, 4638, 2006, 8038, 1036, 2222, 1999, 1037, 2978, 1012, 10047, 1999, 2005, 1037, 2388, 1036, 1055, 2154, 2606, 12690, 1012, 9915, 2080, 999, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Структура токенизированного обучающего набора: Dataset({
    features: ['id', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 500
})
Структура токенизированного тестового набора: Dataset({
    features: ['id', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 200
})


In [8]:
# --- 4. Model Initialization ---
print("\n--- Инициализация модели ---")
# num_labels=3 потому что у нас 3 категории тональности (позитивная, нейтральная, негативная)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

print(f"Модель {model_name} успешно инициализирована для {model.num_labels} меток.")


--- Инициализация модели ---


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель distilbert-base-uncased успешно инициализирована для 3 меток.


In [11]:
import evaluate
# --- 5. Evaluation Setup ---
print("\n--- Настройка оценки ---")
# Устанавливаем метрику 'accuracy'
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred # logits - это "сырые" предсказания модели, labels - истинные метки
    predictions = np.argmax(logits, axis=-1) # Выбираем класс с наибольшим логитом
    return accuracy.compute(predictions=predictions, references=labels)

print("Метрика 'accuracy' и функция compute_metrics настроены.")



--- Настройка оценки ---


Метрика 'accuracy' и функция compute_metrics настроены.


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate # Не забудьте импортировать evaluate, numpy, pandas, os, shutil, если используете весь код
import numpy as np
import os
import shutil
import pandas as pd # Добавлен pandas
# ... (предполагается, что весь остальной код для загрузки, токенизации и загрузки модели присутствует и работает) ...
# Пример загрузки модели, если она не была загружена в предыдущих шагах:
# num_labels = 3
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)
# И, конечно, убедитесь, что train_tokenized_data, test_tokenized_data, compute_metrics определены.

print("\n--- Начинается процесс дообучения ---")

# Аргументы обучения
training_args = TrainingArguments(
    output_dir="./results",
    # ИЗМЕНЕНО ЗДЕСЬ: с evaluation_strategy на eval_strategy
    eval_strategy="epoch", # Оценивать модель после каждой эпохи
    save_strategy="epoch", # Сохранять модель после каждой эпохи
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    fp16=False if not torch.cuda.is_available() else True,
    report_to="none",
    save_total_limit=1
)

# Настройка Trainer
# Убедитесь, что 'model', 'train_tokenized_data', 'test_tokenized_data', 'compute_metrics' определены
trainer = Trainer(
    model=model, # Убедитесь, что модель инициализирована ранее
    args=training_args,
    train_dataset=train_tokenized_data, # Убедитесь, что train_tokenized_data инициализирована ранее
    eval_dataset=test_tokenized_data, # Убедитесь, что test_tokenized_data инициализирована ранее
    compute_metrics=compute_metrics, # Убедитесь, что compute_metrics определена ранее
)

# Тренировка модели
trainer.train()

print("\n--- Обучение завершено. Оценка результатов ---")

# Оценка результатов
results = trainer.evaluate()
print(f"\nРезультаты оценки на тестовом наборе: {results}")

print("\n--- Код выполнен. ---")


--- Начинается процесс дообучения ---


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.805038,0.585000
2,No log,0.621889,0.750000



--- Обучение завершено. Оценка результатов ---



Результаты оценки на тестовом наборе: {'eval_loss': 0.6218894720077515, 'eval_accuracy': 0.75, 'eval_runtime': 22.0705, 'eval_samples_per_second': 9.062, 'eval_steps_per_second': 1.133, 'epoch': 2.0}

--- Код выполнен. ---
